<font color="#CA3532"><h1 align="left">**Aprendizaje por Refuerzo**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Multiarmed Bandits Contextuales**</h2></font>

In [ ]:
# Adaptado de:
# https://github.com/fellowship/space-bandits/releases/tag/v0.0.9

In [ ]:
COLAB = True

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1ZEYZXdC7GyHXqnt5Wl_orkHmc97vWfGO',
                                        dest_path='./spacebandits.zip', unzip=True)
    gdd.download_file_from_google_drive(file_id='1IM_OIFEIhG9vc4Ldg7q4-JG_BZpog4as',
                                        dest_path='./historico_clientes_problema1.csv')
    gdd.download_file_from_google_drive(file_id='1mSlAdoi6cv2QaUy676LZSpRYdd8ncJHc',
                                        dest_path='./simula_clientes_problema1.py')

In [ ]:
!ls

In [ ]:
import numpy as np
import pandas as pd
from random import random, randint
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

# **Aprendizaje sobre base de datos histórica (donde se realizaron acciones aleatorias)**

In [ ]:
df = pd.read_csv("./historico_clientes_problema1.csv")
print(df.shape)
#df[df["accion"]==0]
df

In [ ]:
df["reward"].mean()

In [ ]:
#df[df["reward"]==25]

In [ ]:
n_acciones = df["accion"].nunique()
n_acciones

In [ ]:
n_features = 2 # edad e ingresos_generados (IG)

In [ ]:
def plot_decision_boundary(model, X, Y, h=1, scale=1., parallelize=True,
                           title='decision boundary', thompson=False,
                           classic=False, n_threads=-1, flip_colors=True,
                           alpha=0.1):
    ftnames = X.columns[0], X.columns[1]
    X = X.values
    #model.fit(X[:, :2], Y)
    x_min = X[:, 0].min() - .5
    x_max = X[:, 0].max() + .5
    y_min = X[:, 1].min() - .5
    y_max = X[:, 1].max() + .5
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h)
    )
    if classic:
        Z = model.classic_predict(np.c_[xx.ravel(), yy.ravel()]/scale, thompson=thompson)
    else:
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()]/scale,
                          thompson=thompson,
                          parallelize=parallelize)

    # Put the result into a color plot.
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, alpha=.25)

    # Add the training points to the plot.
    if flip_colors:
        Y = np.where(np.array(Y)==1, 0, 1)
    plt.scatter(X[:, 0], X[:, 1], c=Y, alpha=alpha);
    #plt.scatter(X[:, 0], X[:, 1], c='black', alpha=.1);


    plt.xlabel(ftnames[0], fontsize=14)
    plt.ylabel(ftnames[1], fontsize=14)

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max());
    plt.title(title)
    plt.show()

In [ ]:
def plot_decision_boundary_simpl(model, X, h=1, scale=1., parallelize=True,
                                 title='decision boundary', thompson=False,
                                 classic=False, n_threads=-1, flip_colors=True,
                                 alpha=0.1):
    ftnames = X.columns[0], X.columns[1]
    X = X.values
    x_min = X[:, 0].min() - .5
    x_max = X[:, 0].max() + .5
    y_min = X[:, 1].min() - .5
    y_max = X[:, 1].max() + .5
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h)
    )
    if classic:
        Z = model.classic_predict(np.c_[xx.ravel(), yy.ravel()]/scale, thompson=thompson)
    else:
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()]/scale,
                          thompson=thompson,
                          parallelize=parallelize)

    # Put the result into a color plot.
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, alpha=.25)

    plt.xlabel(ftnames[0], fontsize=14)
    plt.ylabel(ftnames[1], fontsize=14)

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max());
    plt.title(title)
    plt.show()

### **Preparación de los datos para entrenamiento**

Como se puede ver arriba las filas de datos deben organizarse en tripetas: características (features), acciones y recompensas.
En nuestro caso cada registro representa un cliente: sus características (edad, ingresos generados), acción (qué promoción se les ofreció) y la ganancia (cuánto dinero gastó el cliente en la promoción, 0 si no se acogió a la promoción ofrecida).

In [ ]:
from space_bandits import LinearBandits

In [ ]:
df.head(5)

In [ ]:
#split data into triplets
contextos = df[['edad', 'ingresos_generados']]
acciones  = df['accion']
rewards   = df['reward']

In [ ]:
contextos.head()

In [ ]:
acciones.head()

In [ ]:
rewards.head()

In [ ]:
modelo_lineal = LinearBandits(n_acciones, n_features,
                              initial_pulls=100)

#call .fit method; num_updates will repeat training n times
print("Entrenando modelo ...")
# Entreno el modelo sobre resultados históricos:
modelo_lineal.fit(contextos, acciones, rewards, num_updates=10)

plot_decision_boundary(modelo_lineal, contextos, acciones, h=.5,
                       thompson=False, scale=1.,
                       title='Modelo + acciones tomadas en datos históricos')

In [ ]:
plot_decision_boundary_simpl(modelo_lineal, contextos, h=.5,
                             scale=1., title='Frontera de decisión aprendida')

In [ ]:
acciones_ = modelo_lineal.predict(contextos.values, thompson=False)
plot_decision_boundary(modelo_lineal, contextos, acciones_, h=.5,
                       thompson=False, scale=1.,flip_colors=False,
                       title='Modelo + acciones que se deberían haber tomado según el modelo')

In [ ]:
# se graba el modelo construido
modelo_lineal.save('model1.pkl')

La base de datos histórica ha sido construida asignando acciones al azar. La ganancia promedio por cliente es:

In [ ]:
print("Ganancia promedio en base de datos histórica:")
df["reward"].mean()

# **Explotación**

Ahora usaremos el modelo entrenado para predecir nuevos clientes (explotación):

In [ ]:
from space_bandits import load_linear_model
from simula_clientes_problema1 import cliente

modelo_lineal = load_linear_model("model1.pkl")

In [ ]:
def show_records(records):
    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    cummulative_reward_plt = np.cumsum(records["reward_record"])
    ts = records['timesteps']
    plt.plot(ts, cummulative_reward_plt, label='cummulative reward')
    plt.plot([ts[0],ts[-1]], [0,0], ':k')
    plt.title("Total reward: {}".format(cummulative_reward_plt[-1]))
    plt.legend()

    plt.subplot(1,2,2)
    avg_reward_plt = np.cumsum(records["reward_record"]).astype(float)
    for i in range(len(avg_reward_plt)):
        avg_reward_plt[i] /= (i+1)
    plt.plot(ts, avg_reward_plt, label='average reward por cliente')
    plt.title("Average Reward: {}".format(avg_reward_plt[-1]))
    plt.legend()
    plt.show()

def explotacion(modelo, steps, records = None,
                plot_frequency=250, avg_length=150,
                random_state=None, update=True):

    if random_state is not None:
        np.random.seed(random_state)

    #these will track values for plotting
    if records is None:
        records = dict()
        records['timesteps'] = []
        records['reward_record'] = []
        start = 0
    else:
        start = records['timesteps'][-1] + 1
    for i in range(start, start+steps):
        records['timesteps'].append(i)

        c = cliente() # cliente que llega

        contexto = c.get_context() # datos contextuales del cliente
        accion = modelo.action(contexto) # acción recomendada por el modelo
        reward = c.get_reward(accion) # ganancia tras lanzar acción
        if update:
            modelo.update(contexto, accion, reward) # actualización del modelo

        records['reward_record'].append(reward)

        if (i+1) % plot_frequency == 0 and i > 0:
            show_records(records)
            plt.show()
    return records

In [ ]:
records = explotacion(modelo_lineal, steps=2000, plot_frequency=500, update=False)

In [ ]:
records = explotacion(modelo_lineal, steps=2000, plot_frequency=500, update=True)

### **Interpretación del modelo (XAI): ¿qué relación hay entre las variables de contexto y las recomendaciones realizadas por el modelo?**

In [ ]:
# Técnica del modelo surrogado:
# Construyo un modelo que hace lo mismo que mi modelo caja negra,
# pero es interpretable

# Si tienes un modelo que explicar, y el modelo predice y_pred = MODELO(x)
# Explicación con modelo surrogado: entrena árbol cuyo target sea y_pred, y cuyo input sea x

# y_pred = MODELO(x) # (modelo caja negra)
# yo quiero obtener una explicación a cómo decide MODELO
# arbol.fit(x', y_pred)

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1uvqBzz-WnK5MWezXiO9htr54DWSxHWsL',
                                        dest_path="./libreria_aux_arboles.py.txt")
    !mv ./libreria_aux_arboles.py.txt ./libreria_aux_arboles.py
    gdd.download_file_from_google_drive(file_id='1u7-C8oy2NbzDtKVP3quUlXB7LLSkHc_d',
                                        dest_path="./my_library.py.txt")
    !mv ./my_library.py.txt ./my_library.py

In [ ]:
df

In [ ]:
contextos

In [ ]:
contextos.values

In [ ]:
# acciones predichas por mi modelo caja negra:
acciones_predichas = [modelo_lineal.action(contexto) for contexto in contextos.values]

In [ ]:
len(acciones_predichas)

In [ ]:
df["acciones_predichas"] = acciones_predichas

In [ ]:
df

##IA Explicable: interpretación del modelo de aprendizaje por refuerzo

Técnica del modelo subrogado

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from libreria_aux_arboles import tree_to_code, tree_to_pseudo
from graphviz import Source

startbold = '\033[1m'
endbold = '\033[0m'

clf = DecisionTreeClassifier(
    #max_depth=1,
    min_samples_split=0.3, # mínimo número de casos en un nodo para partirlo. 0.3 = 30% del total de casos
    min_samples_leaf=0.2,  # mínimo número de casos en nodo hoja. 0.2 = 20% del total de casos
    max_leaf_nodes=5, # máximo número de nodos hoja
    #min_weight_fraction_leaf=0.05,
    #min_impurity_decrease=0.15
    )

clf = clf.fit(contextos, acciones_predichas)
# clf.classes_
attributes_names = df.columns[:2]

tree_to_code(clf, attributes_names, start_bold=startbold, end_bold=endbold)

display(Source( export_graphviz(clf, out_file=None,
                                feature_names=attributes_names,
                                #class_names=cluster_names,
                                filled=True, rounded=True,
                                special_characters=True,
                                impurity=True,
                                leaves_parallel=True,
                                rotate=False,
                                node_ids=True)))

In [ ]:
clf.classes_

In [ ]:
clf.score(contextos, acciones_predichas)

In [ ]:
from my_library import get_rules_from_tree
action_names = ["recomendar producto {}".format(x) for x in clf.classes_]
rules = get_rules_from_tree(clf, attributes_names, action_names, contextos.values, np.array(acciones_predichas))

print(len(rules), "rules:\n")
for item in rules.items():
    print('\033[1m' + "* ", item[1][0], '\033[0m')
    for c in item[1][1]:
        print("     "+c[0]+":", c[1], "cases (%.2f%%)" % (100*c[2]))